In [1]:
import os
from transformers import BertTokenizer, BertForSequenceClassification
import numpy as np
import random
from torch.utils.data import DataLoader
from torch.optim import AdamW
from torch import nn
from transformers import get_linear_schedule_with_warmup
from tqdm import tqdm
from peft import get_peft_model, PrefixTuningConfig, TaskType, LoraConfig
from utils import *

In [2]:
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
device = torch.device("cuda:0")

In [3]:
dataset_name = "sst2" #sst2, qnli qqp mnli
tokenized_data = load_cleaned_data(dataset_name)
train_dataloader = DataLoader(tokenized_data['train'], shuffle=False, batch_size=1024,collate_fn=collate_fn)
if dataset_name == "mnli":
    tokenized_data['validation'] = tokenized_data["validation_matched"]
val_dataloader = DataLoader(tokenized_data['validation'], shuffle=False, batch_size=1024,collate_fn=collate_fn)
num_labels = torch.unique(tokenized_data["train"]["labels"]).numel()
loss_fn = nn.CrossEntropyLoss()

Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Tue Mar 25 23:46:02 2025) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.
03/31/2025 23:39:00:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/datasets/glue/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on Tue Mar 25 23:46:02 2025) since it couldn't be found locally at glue., or remotely on the Hugging Face Hub.


Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

# Prefix without DP

In [4]:
model_name = "prajjwal1/bert-tiny"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

for param in model.parameters():
    param.requires_grad = False
    
prefix_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=20,
)
model = get_peft_model(model, prefix_config)
model.to(device)

optimizer = AdamW(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
epochs = 5

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * epochs),)

model.print_trainable_parameters()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 10,498 || all params: 4,396,676 || trainable%: 0.2388


In [5]:
trainModel(model,optimizer,train_dataloader,val_dataloader,loss_fn,lr_scheduler,tqdm,dataset_name)

100%|██████████| 66/66 [00:07<00:00,  8.67it/s]
/media/nvidia/00ad733b-135d-4406-9b4f-61c0260a5b45/project/SWX/tiny-bert/utils.py:72: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", task)


epoch=0: train_ppl=tensor(1.9145, device='cuda:0') train_epoch_loss=tensor(0.6494, device='cuda:0') 


Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.
03/31/2025 23:39:30:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.


Epoch 1, Validation Accuracy without DP: {'accuracy': 0.6662844036697247}


100%|██████████| 66/66 [00:06<00:00,  9.78it/s]


...

Epoch 4, Validation Accuracy without DP: {'accuracy': 0.7075688073394495}


100%|██████████| 66/66 [00:06<00:00,  9.83it/s]


epoch=4: train_ppl=tensor(1.7680, device='cuda:0') train_epoch_loss=tensor(0.5698, device='cuda:0') 


Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.
03/31/2025 23:40:11:WARNING:Using the latest cached version of the module from /home/nvidia/.cache/huggingface/modules/datasets_modules/metrics/glue/91f3cfc5498873918ecf119dbf806fb10815786c84f41b85a5d3c47c1519b343 (last modified on Fri Mar 28 18:34:17 2025) since it couldn't be found locally at glue, or remotely on the Hugging Face Hub.


Epoch 5, Validation Accuracy without DP: {'accuracy': 0.7155963302752294}
Training complete!


# Prefix with DP

In [6]:
model_name = "prajjwal1/bert-tiny"
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

for param in model.parameters():
    param.requires_grad = False
    
prefix_config = PrefixTuningConfig(
    task_type=TaskType.SEQ_CLS,
    num_virtual_tokens=20,
)
model = get_peft_model(model, prefix_config)
model.to(device)

optimizer = AdamW(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()
epochs = 5

lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * epochs),)

model.print_trainable_parameters()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 10,498 || all params: 4,396,676 || trainable%: 0.2388


In [7]:
dp_train(model,train_dataloader,tokenized_data,optimizer,lr_scheduler,epochs,val_dataloader,dataset_name)

/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/opacus/privacy_engine.py:142: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  warnings.warn(
/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/opacus/accountants/analysis/rdp.py:332: UserWarning: Optimal order is the largest alpha. Please consider expanding the range of alphas to get a tighter privacy bound.
  warnings.warn(
/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/opacus/accountants/analysis/prv/prvs.py:50: RuntimeWarning: invalid value encountered in log
  z = np.log((np.exp(t) + q - 1) / q)
  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autogra

Epoch 1, Validation Accuracy: {'accuracy': 0.533256880733945}


  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1082: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
100%|██████████| 66/66 [00:06<00:00,

Epoch 2, Validation Accuracy: {'accuracy': 0.6077981651376146}


  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1082: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
100%|██████████| 66/66 [00:10<00:00,

Epoch 3, Validation Accuracy: {'accuracy': 0.6318807339449541}


  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1082: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
100%|██████████| 66/66 [00:09<00:00,

Epoch 4, Validation Accuracy: {'accuracy': 0.6387614678899083}


  0%|          | 0/66 [00:00<?, ?it/s]/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1117: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/home/nvidia/anaconda3/envs/datum/lib/python3.8/site-packages/torch/nn/modules/module.py:1082: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
100%|██████████| 66/66 [00:09<00:00,

Epoch 5, Validation Accuracy: {'accuracy': 0.643348623853211}
Training complete
